# Notebook with Example Config for Different Models / Hooks

## Setup

In [1]:
import torch
import os 
import sys
sys.path.append("..")
from sae_training.config import LanguageModelSAERunnerConfig
from sae_training.lm_runner import language_model_sae_runner

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("Using device:", device)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Using device: cuda


## Gelu-2L

An example of a toy language model we're able to train on.

### MLP Out

In [2]:


for expansion_factor in [8]:
    for finetuning_method in ["decoder"]:
        cfg = LanguageModelSAERunnerConfig(

            # Data Generating Function (Model + Training Distibuion)
            model_name = "gelu-2l",
            hook_point = "blocks.0.hook_mlp_out",
            hook_point_layer = 0,
            d_in = 512,
            dataset_path = "NeelNanda/c4-tokenized-2b",
            is_dataset_tokenized=True,
            
            # SAE Parameters
            expansion_factor = expansion_factor,
            b_dec_init_method="mean", # geometric median is better but slower to get started
            normalize_activations = True,
            use_pre_encoder_bias= False,
            
            # Training Parameters
            lr = 0.0012,
            lr_scheduler_name="constantwithwarmup",
            l1_coefficient = 0.004,
            train_batch_size = 4096,
            context_size = 1024,
            adam_beta1 = 0,
            adam_beta2 = 0.9999,
            finetuning_method = finetuning_method,
            
            # Activation Store Parameters
            n_batches_in_buffer = 128,
            total_training_tokens = 1_000_000 * 100, 
            fine_tune_tokens = 1_000_000 * 100,
            store_batch_size = 32,
            
            # Resampling protocol
            mse_loss_normalization = "variance",
            ghost_grads=None,#"residual",
            feature_sampling_window = 500,
            dead_feature_window=500,
            dead_feature_threshold = 1e-5,
            
            # WANDB
            log_to_wandb = True,
            wandb_project= "mats_sae_training_language_models_gelu_2l_finetuning_experiment",
            wandb_log_frequency=10,
            
            # Misc
            device = device,
            seed = 42,
            n_checkpoints = 0,
            checkpoint_path = "checkpoints",
            dtype = torch.bfloat16,
            )


        sparse_autoencoder = language_model_sae_runner(cfg)


Run name: 4096-L1-0.004-LR-0.0012-Tokens-1.000e+08
n_tokens_per_buffer (millions): 4.194304
Lower bound: n_contexts_per_buffer (millions): 0.004096
Total training steps: 24414
Total wandb updates: 2441
n_tokens_per_feature_sampling_window (millions): 2097.152
n_tokens_per_dead_feature_window (millions): 2097.152
We will reset the sparsity calculation 48 times.
Number tokens in sparsity calculation window: 2.05e+06


Loaded pretrained model gelu-2l into HookedTransformer
Moving model to device:  cuda


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jbloom. Use `wandb login --relogin` to force relogin


Reinitializing b_dec with mean of activations
Previous distances: 13.375
New distances: 10.25


4598| MSE Loss 0.116 | L1 0.035:   9%|▉         | 18833408/200000000 [04:46<14:47, 204121.53it/s] 

KeyboardInterrupt: 

4598| MSE Loss 0.116 | L1 0.035:   9%|▉         | 18837504/200000000 [05:00<14:47, 204121.53it/s]

## GPT2 - Small

### Residual Stream

In [ ]:
from sae_training.config import LanguageModelSAERunnerConfig
from sae_training.lm_runner import language_model_sae_runner

layer = 3
cfg = LanguageModelSAERunnerConfig(

    # Data Generating Function (Model + Training Distibuion)
    model_name = "gpt2-small",
    hook_point = f"blocks.{layer}.hook_resid_pre",
    hook_point_layer = layer,
    d_in = 768,
    dataset_path = "Skylion007/openwebtext",
    is_dataset_tokenized=False,
    
    # SAE Parameters
    expansion_factor = 32, # determines the dimension of the SAE.
    b_dec_init_method = "mean", # geometric median is better but slower to get started
    
    # Training Parameters
    lr = 0.0004,
    l1_coefficient = 0.00008,
    lr_scheduler_name="constantwithwarmup",
    train_batch_size = 4096,
    context_size = 128,
    lr_warm_up_steps=5000,
    
    # Activation Store Parameters
    n_batches_in_buffer = 128,
    total_training_tokens = 1_000_000 * 300, # 200M tokens seems doable overnight.
    store_batch_size = 32,
    
    # Resampling protocol
    use_ghost_grads=True,
    feature_sampling_window = 2500,
    dead_feature_window=5000,
    dead_feature_threshold = 1e-8,
    
    # WANDB
    log_to_wandb = True,
    wandb_project= "mats_sae_training_language_models_resid_pre_test",
    wandb_entity = None,
    wandb_log_frequency=100,
    
    # Misc
    device = "cuda",
    seed = 42,
    n_checkpoints = 10,
    checkpoint_path = "checkpoints",
    dtype = torch.float32,
    )

sparse_autoencoder = language_model_sae_runner(cfg)


# Pythia 70-M

In [ ]:
import torch
import os 
import sys 

sys.path.append("..")
from sae_training.config import LanguageModelSAERunnerConfig
from sae_training.lm_runner import language_model_sae_runner

import cProfile


os.environ["TOKENIZERS_PARALLELISM"] = "false"
cfg = LanguageModelSAERunnerConfig(

    # Data Generating Function (Model + Training Distibuion)
    model_name = "pythia-70m-deduped",
    hook_point = "blocks.0.hook_mlp_out",
    hook_point_layer = 0,
    d_in = 512,
    dataset_path = "EleutherAI/the_pile_deduplicated",
    is_dataset_tokenized=False,
    
    # SAE Parameters
    expansion_factor = 64,
    
    # Training Parameters
    lr = 3e-4,
    l1_coefficient = 4e-5,
    train_batch_size = 8192,
    context_size = 128,
    lr_scheduler_name="constantwithwarmup",
    lr_warm_up_steps=10_000,
    
    # Activation Store Parameters
    n_batches_in_buffer = 64,
    total_training_tokens = 1_000_000 * 800, 
    store_batch_size = 32,
    
    # Resampling protocol
    feature_sampling_method = 'anthropic',
    feature_sampling_window = 2000, # Doesn't currently matter.
    feature_reinit_scale = 0.2,
    dead_feature_window=40000,
    dead_feature_threshold = 1e-8,
    
    # WANDB
    log_to_wandb = True,
    wandb_project= "mats_sae_training_language_benchmark_tests",
    wandb_entity = None,
    wandb_log_frequency=20,
    
    # Misc
    device = "cuda",
    seed = 42,
    n_checkpoints = 0,
    checkpoint_path = "checkpoints",
    dtype = torch.float32,
    )


sparse_autoencoder = language_model_sae_runner(cfg)


# Pythia 70M Hook Q

In [ ]:
import torch
import os 
import sys
sys.path.append("../")

from sae_training.config import LanguageModelSAERunnerConfig
from sae_training.lm_runner import language_model_sae_runner

os.environ["TOKENIZERS_PARALLELISM"] = "false"
cfg = LanguageModelSAERunnerConfig(

    # Data Generating Function (Model + Training Distibuion)
    model_name = "pythia-70m-deduped",
    hook_point = "blocks.2.attn.hook_q",
    hook_point_layer = 2,
    hook_point_head_index=7,
    d_in = 64,
    dataset_path = "EleutherAI/the_pile_deduplicated",
    is_dataset_tokenized=False,
    
    # SAE Parameters
    expansion_factor = 16,
    
    # Training Parameters
    lr = 0.0012,
    l1_coefficient = 0.003,
    lr_scheduler_name="constantwithwarmup",
    lr_warm_up_steps=1000, # about 4 million tokens.
    train_batch_size = 4096,
    context_size = 128,
    
    # Activation Store Parameters
    n_batches_in_buffer = 128,
    total_training_tokens = 1_000_000 * 1500,
    store_batch_size = 32,
    
    # Resampling protocol
    feature_sampling_method = 'anthropic',
    feature_sampling_window = 1000,# doesn't do anything currently.
    feature_reinit_scale = 0.2,
    resample_batches=8,
    dead_feature_window=60000,
    dead_feature_threshold = 1e-5,
    
    # WANDB
    log_to_wandb = True,
    wandb_project= "mats_sae_training_pythia_70M_hook_q_L2H7",
    wandb_entity = None,
    wandb_log_frequency=100,
    
    # Misc
    device = "mps",
    seed = 42,
    n_checkpoints = 15,
    checkpoint_path = "checkpoints",
    dtype = torch.float32,
    )

sparse_autoencoder = language_model_sae_runner(cfg)


# Tiny Stories

In [ ]:
import torch
import os 

from sae_training.config import LanguageModelSAERunnerConfig
from sae_training.lm_runner import language_model_sae_runner



os.environ["TOKENIZERS_PARALLELISM"] = "false"
cfg = LanguageModelSAERunnerConfig(

    # Data Generating Function (Model + Training Distibuion)
    model_name = "tiny-stories-2L-33M",
    hook_point = "blocks.1.mlp.hook_post",
    hook_point_layer = 1,
    d_in = 4096,
    dataset_path = "roneneldan/TinyStories",
    is_dataset_tokenized=False,
    
    # SAE Parameters
    expansion_factor = 4,
    
    # Training Parameters
    lr = 1e-4,
    l1_coefficient = 3e-4,
    train_batch_size = 4096,
    context_size = 128,
    
    # Activation Store Parameters
    n_batches_in_buffer = 128,
    total_training_tokens = 1_000_000 * 10, # want 500M eventually.
    store_batch_size = 32,
    
    # Resampling protocol
    feature_sampling_method = 'l2',
    feature_sampling_window = 2500, # Doesn't currently matter.
    feature_reinit_scale = 0.2,
    dead_feature_window=1250,
    dead_feature_threshold = 0.0005,
    
    # WANDB
    log_to_wandb = True,
    wandb_project= "mats_sae_training_language_benchmark_tests",
    wandb_entity = None,
    wandb_log_frequency=10,
    
    # Misc
    device = "mps",
    seed = 42,
    n_checkpoints = 0,
    checkpoint_path = "checkpoints",
    dtype = torch.float32,
    )

sparse_autoencoder = language_model_sae_runner(cfg)


# Toy Model

In [ ]:

from sae_training.toy_model_runner import SAEToyModelRunnerConfig, toy_model_sae_runner


cfg = SAEToyModelRunnerConfig(
    
    # Model Details
    n_features=200,
    n_hidden=5,
    n_correlated_pairs=0,
    n_anticorrelated_pairs=0,
    feature_probability=0.025,
    model_training_steps=10_000,
    
    # SAE Parameters
    d_sae=240,
    l1_coefficient=0.001,
    
    # SAE Train Config
    train_batch_size=1028,
    feature_sampling_window=3_000,
    dead_feature_window=1_000,
    feature_reinit_scale=0.5,
    total_training_tokens=4096*300,
    
    # Other parameters
    log_to_wandb=True,
    wandb_project="sae-training-test",
    wandb_log_frequency=5,
    device="mps",
)

trained_sae = toy_model_sae_runner(cfg)

assert trained_sae is not None


# Run caching of activations to disk

In [ ]:
import torch
import os 
import sys
sys.path.append("..")
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB__SERVICE_WAIT"] = "300"

from sae_training.config import CacheActivationsRunnerConfig
from sae_training.cache_activations_runner import cache_activations_runner

cfg = CacheActivationsRunnerConfig(

    # Data Generating Function (Model + Training Distibuion)
    model_name = "gpt2-small",
    hook_point = f"blocks.{layer}.hook_resid_pre",
    hook_point_layer = 10,
    hook_point_head_index=7,
    d_in = 64,
    dataset_path = "Skylion007/openwebtext",
    is_dataset_tokenized=False,
    cached_activations_path="../activations/",
    
    # Activation Store Parameters
    n_batches_in_buffer = 16,
    total_training_tokens = 500_000_000, 
    store_batch_size = 32,

    # Activation caching shuffle parameters
    n_shuffles_final = 16,
    
    # Misc
    device = "mps",
    seed = 42,
    dtype = torch.float32,
    )

cache_activations_runner(cfg)


## Train an SAE using the cached activations stored on disk
Pass `use_cached_activations=True` into the config

In [ ]:
import torch
import os 
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB__SERVICE_WAIT"] = "300"
from sae_training.config import LanguageModelSAERunnerConfig
from sae_training.lm_runner import language_model_sae_runner

cfg = LanguageModelSAERunnerConfig(

    # Data Generating Function (Model + Training Distibuion)
    model_name = "gpt2-small",
    hook_point = "blocks.10.hook_resid_pre",
    hook_point_layer = 11,
    d_in = 768,
    dataset_path = "Skylion007/openwebtext",
    is_dataset_tokenized=False,
    use_cached_activations=True,
    
    # SAE Parameters
    expansion_factor = 64, # determines the dimension of the SAE.
    
    # Training Parameters
    lr = 1e-5,
    l1_coefficient = 5e-4,
    lr_scheduler_name=None,
    train_batch_size = 4096,
    context_size = 128,
    
    # Activation Store Parameters
    n_batches_in_buffer = 64,
    total_training_tokens = 200_000,
    store_batch_size = 32,
    
    # Resampling protocol
    feature_sampling_method = 'l2',
    feature_sampling_window = 1000,
    feature_reinit_scale = 0.2,
    dead_feature_window=5000,
    dead_feature_threshold = 1e-7,
    
    # WANDB
    log_to_wandb = True,
    wandb_project= "mats_sae_training_gpt2_small",
    wandb_entity = None,
    wandb_log_frequency=50,
    
    # Misc
    device = "mps",
    seed = 42,
    n_checkpoints = 5,
    checkpoint_path = "checkpoints",
    dtype = torch.float32,
    )

sparse_autoencoder = language_model_sae_runner(cfg)


# Scratch

In [ ]:
import torch
import os 
import sys 
sys.path.append("..")
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB__SERVICE_WAIT"] = "300"
from sae_training.config import LanguageModelSAERunnerConfig
from sae_training.lm_runner import language_model_sae_runner



# for l1_coefficient in [9e-4,8e-4,7e-4]:
cfg = LanguageModelSAERunnerConfig(

    # Data Generating Function (Model + Training Distibuion)
    model_name = "gpt2-small",
    hook_point = "blocks.10.attn.hook_q",
    hook_point_layer = 10,
    hook_point_head_index=7,
    d_in = 64,
    dataset_path = "Skylion007/openwebtext",
    is_dataset_tokenized=False,
    use_cached_activations=True,
    cached_activations_path="../activations/",
    
    # SAE Parameters
    expansion_factor = 64, # determines the dimension of the SAE. (64*64 = 4096, 64*4*64 = 32768)
    
    # Training Parameters
    lr = 1e-3,
    l1_coefficient = 2e-4,
    # lr_scheduler_name="LinearWarmupDecay",
    lr_warm_up_steps=2200,
    train_batch_size = 4096,
    context_size = 128,
    
    # Activation Store Parameters
    n_batches_in_buffer = 512,
    total_training_tokens = 3_000_000,
    store_batch_size = 32,
    
    # Resampling protocol
    feature_sampling_method = 'l2',
    feature_sampling_window = 1000,
    feature_reinit_scale = 0.2,
    dead_feature_window=200,
    dead_feature_threshold = 5e-6,
    
    # WANDB
    log_to_wandb = True,
    wandb_project= "mats_sae_training_gpt2_small_hook_q_dev",
    wandb_entity = None,
    wandb_log_frequency=5,
    
    # Misc
    device = "mps",
    seed = 42,
    n_checkpoints = 0,
    checkpoint_path = "checkpoints",
    dtype = torch.float32,
    )

# cfg.d_sae
sparse_autoencoder = language_model_sae_runner(cfg)
# assert sparse_autoencoder is not None